In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import re

import jieba
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import Word2Vec
from sklearn.preprocessing import scale
import multiprocessing

from snownlp import SnowNLP
import jieba.analyse

In [2]:
dff = pd.read_csv("C:/Users/Kai/Desktop/171840708_IntroDM_MiningChallenge/mining-challenge-for-nju-introdm-2019/Mining Challenge Dataset/train.csv",index_col=0)
dff['text'] = dff['text'].fillna('')
dff.head(10)

,text,class,positive
index,,,
0,﻿18年结婚 哈哈哈,0,0.900696
1,2017最后顿大餐吃完两人世界明年就是三个人一起啦许下生日愿望️希望一家人都能顺利平安健康🏻🏻🏻,1,0.999904
2,意盎然的季节！祝愿大家都生机勃勃，郁郁葱葱！,2,0.736431
3,2017 遇见挚友 遇见我老公 结了婚有了小芒果 希望2018也超级美好️,3,0.983905
4,2018.1.1,4,0.500000
5,2018加油！,5,0.895319
6,2018年做一个更加真实的自己。️,3,0.783433
7,2018年的第一天，完美的错过了一辆公交车。 德州,6,0.934181
8,2018年目标1.赚钱买房2.谈场恋爱，遇到对的人就结婚3.拥有一副健康的身体4.学会一种乐...,7,0.999799


In [3]:
def stopwordslist():
    f = open("C:/Users/Kai/Desktop/stop.txt", "r")
    line = f.readline()
    stopwords = []
    index = 0
    while line:
        if index % 1000 == 0:
            print(index)
        index += 1
        line = line.replace('\n', '')
        line = line.replace('[', '')
        line = line.replace(']', '')
        line = line.replace('］', '')
        line = line.replace('［', '')
        
        stopwords.append(line)
        line = f.readline()

    print(stopwords[:100])
    return stopwords

# 创建一个停用词列表
stopwords = stopwordslist()

0
[' ', '  ', '~', '。', '，', '…', '～', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '--', '.', ':', '://', '::', ';', '<', '=', '>', '>>', '?', '@', 'A', 'Lex', '', '\\', '', '^', '_', '`', 'exp', 'sub', 'sup', '|', '}', '~', '~~~~', '·', '×', '×××', 'Δ', 'Ψ', 'γ', 'μ', 'φ', 'φ．', 'В', '—', '——', '———', '‘', '’', '’‘', '“', '”', '”，', '…', '……', '…………………………………………………③', '′∈', '′｜', '℃', 'Ⅲ', '↑', '→', '∈', '∪φ∈', '≈', '①', '②', '②ｃ', '③', '③', '④', '⑤', '⑥', '⑦', '⑧', '⑨', '⑩', '──', '■', '▲', '\u3000', '、', '。', '〈', '〉', '《', '》']


In [4]:
# 对句子进行中文分词
def seg_depart(sentence):
    # 对文档中的每一行进行中文分词
    sentence_depart = jieba.cut(sentence.strip())
    # 输出结果为outstr
    outstr = ''
    # 去停用词
    for word in sentence_depart:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

In [5]:
clas = dff['class'].values

In [6]:
dfTest = pd.read_csv("C:/Users/Kai/Desktop/171840708_IntroDM_MiningChallenge/mining-challenge-for-nju-introdm-2019/Mining Challenge Dataset/test.csv",index_col=0)
dfTest['text'] = dfTest['text'].fillna('')
dfTest.head(10)


,text,class,positive
index,,,
0,我是正面哦,0,0.347826
1,爱是恒久忍耐，又有恩慈。爱是不嫉妒，不自夸，不张狂，不轻易发怒。不计算人的恶。凡事包容。凡事...,0,0.496333
2,讨厌死了，上班上班上班不停的上班我真的超级累。什么都不干还是超级超级累。,0,0.000422
3,矮马大半夜的放肌肉男不让人睡觉了,0,0.409895
4,谢谢陈先生。,0,0.768959
5,我的2016要早点睡别熬夜,0,0.625607
6,周锐锐哥！爱你,0,0.970187
7,塞尼亚岛,0,0.500000
8,只可惜没能去现场,0,0.100791


In [7]:
# 分词
sen = dff['text'].values

for i in range(len(sen)):
    if i % 100000 == 0:
        print(i)
    sen[i] = seg_depart(sen[i])
    


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Kai\AppData\Local\Temp\jieba.cache


0


Loading model cost 0.879 seconds.
Prefix dict has been built succesfully.


100000
200000
300000
400000
500000
600000
700000
800000


In [8]:
senTest = dfTest['text'].values

for i in range(len(senTest)):
    if i % 100000 == 0:
        print(i)
    senTest[i] = seg_depart(senTest[i])

0
100000


In [9]:
print(senTest[:30])

['我 是 正面 哦 '
 '爱是 恒久 忍耐 又 有恩慈 爱是 不嫉妒 不 自夸 不 张狂 不 轻易 发怒 不 计算 人 的 恶 凡事 包容 凡事 相信 凡事 盼望 凡事 忍耐 爱是 永不 止息 '
 '讨厌 死 了 上班 上班 上班 不停 的 上班 我 真的 超级 累 什么 都 不 干 还是 超级 超级 累 '
 '矮马 大半夜 的 放 肌肉男 不让 人 睡觉 了 ' '谢谢 陈先生 ' '我 的 2016 要 早点 睡别 熬夜 ' '周锐 锐哥 爱 你 '
 '塞 尼亚岛 ' '只 可惜 没能 去 现场 ' '自从 发现 这个 号 都 处于 一种 忍不住 不 看看 了 睡不着 的 状态 '
 '真系 咁 钟意 音乐 咩 '
 '感恩 2 续 他们 都 会 过 得 很 幸福 甜蜜 爸爸 的 身体 也 越来越 健壮 健康 妈妈 也 越来越 温柔 越 女人 我 自己 也 越来越 漂亮 皮肤 好好 非常 水润 皮肤 非常 光滑 我 弟弟 也 越来越 帅 越来越 思想 成熟 做事 非常 稳重 也 越来越 让 家人 开心 在 南昌 明年 一定 会 有 到 我 的 单身公寓 我 明年 一定 会 拿到 我 的 粉车 '
 '迷尚 自然 的 主页 ' '问叹 女王 权杖 口红 我 最 爱 的 口红 是 口红 又 是 装饰品 '
 '有个 顺序 得 先 读书 然后 才能 多 走走 否则 行再 多路 也 是 个 邮差 音乐 也 是 一样 我 倒 是 也 想 施施然 上台 去 可是 要 被 踹 下来 的 呀 预祝 巡演 成功 '
 '年终 福利 ' '声音 好好 听 '
 '少年 迪玛希 谁家 翩翩少年 郎 横空出世 迷人眼 着 调 专访 少年 迪玛希 谁家 翩翩少年 郎 横空出世 迷人眼 着 调 专访 '
 '喜欢 的 紫薯 甜品店 来 了 ' '我 不是 好惹 的 第 12 名 ' '一天 比 一天 像 公主 梦 都 被 满足 '
 '有 你 在 身边 很 心安 去 校医 室 有人 陪 去 体检 有人 陪 干什么 你 都 在 很快 又 不累 '
 '果然 全世界 女孩子 都 是 一样 的 ... 这 看 脸 的 世界 '
 '11 月 7 日 20 00 上 新 开拍 亲们 来 捧场 哦 上 新 当晚 有 给 力 优惠 还有 神秘 福袋 哦 '
 '吉林 百嘉 门将 原 国家 沙

In [10]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(sen, clas, test_size=0.1, random_state=42)   ######

In [11]:
# vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5)
# transformer = TfidfTransformer()

In [12]:
# tf_X_train = vectorizer.fit_transform(X_train)

In [13]:
# tf_X_test = vectorizer.transform(X_test)

In [14]:
# tf_Test = vectorizer.transform(senTest)

In [15]:
# from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
# maxxi = 0
# maxxscore = 0
# for i in np.arange(10, 20, 0.5):
#     mnb = ComplementNB(alpha=i)
#     mnb.fit(tf_X_train, y_train)
#     print(mnb.score(tf_X_test,y_test), i)
#     if maxxscore < mnb.score(tf_X_test,y_test):
#         maxxscore = mnb.score(tf_X_test,y_test)
#         maxxi = i

# print(maxxscore, maxxi)

In [16]:
# mnb = ComplementNB(alpha=11.5)
# mnb.fit(tf_X_train, y_train)
# print(mnb.score(tf_X_test,y_test), 0.1)


In [17]:
# pred = mnb.predict(tf_Test)

In [18]:
# csvFile = open('C:/Users/Kai/Desktop/171840708_IntroDM_MiningChallenge/mining-challenge-for-nju-introdm-2019/Mining Challenge Dataset/last0.csv','w', newline='', encoding='UTF-8') # 设置newline，否则两行之间会空一行
# writer = csv.writer(csvFile)

# writer.writerow(['ID', 'Expected'])
# for i in range(len(pred)):
#     if i % 50000 == 0:
#         print(i)
#     writer.writerow([int(i), int(pred[i])])
    
# csvFile.close()

In [19]:
# from sklearn.svm import LinearSVC
# model = LinearSVC(penalty='l1', dual=False, tol=1e-3)
# model.fit(tf_X_train, y_train)

In [20]:
# print(model.score(tf_X_test,y_test))

In [21]:
# libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
np.random.seed(32)


from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Conv1D, MaxPooling1D, Dropout
from keras.utils.np_utils import to_categorical


%matplotlib inline
MAX_NB_WORDS = 20000
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, char_level=False)

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
E:\Anaconda3\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [22]:
tokenizer.fit_on_texts(sen)

In [23]:
sequences = tokenizer.texts_to_sequences(sen)

In [24]:
sequences_test = tokenizer.texts_to_sequences(senTest)

In [25]:
MAX_SEQUENCE_LENGTH = 300

# pad sequences with 0s
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)

In [26]:
one_hot = pd.get_dummies(dff["class"])
dff.drop(['class'], axis=1, inplace=True)
dff = pd.concat([dff,one_hot], axis=1)
dff.head()

,text,positive,0,1,2,3,4,5,6,7,...,62,63,64,65,66,67,68,69,70,71
index,,,,,,,,,,,,,,,,,,,,,
0,﻿ 18 年 结婚 哈哈哈,0.900696,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2017 最后 顿 大餐 吃 完 两人 世界 明年 就是 三个 人 一起 啦 许下 生日 愿...,0.999904,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,意 盎然 的 季节 祝愿 大家 都 生机勃勃 郁郁葱葱,0.736431,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2017 遇见 挚友 遇见 我 老公 结了婚 有 了 小 芒果 希望 2018 也 超级 美...,0.983905,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2018.1 1,0.500000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# a, b, yTrain, yTest = train_test_split(sen, y_train, test_size=0.1, random_state=42)   ######

In [27]:
y_train = dff.drop(['text', 'positive'],axis=1).values
print(y_train)

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# 下面的cell是 分数为 0.7+ 的模型源代码


In [28]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding
from keras.models import Model

EMBEDDING_DIM = 100
N_CLASSES = 72

# input: a sequence of MAX_SEQUENCE_LENGTH integers
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)

average = GlobalAveragePooling1D()(embedded_sequences)
predictions = Dense(N_CLASSES, activation='softmax')(average)

model = Model(sequence_input, predictions)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['acc'])

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 100)          2000000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 72)                7272      
Total params: 2,007,272
Trainable params: 2,007,272
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(x_train, y_train, validation_split=0.1, epochs=2, batch_size=128)

Train on 776847 samples, validate on 86317 samples
Epoch 1/2
776847/776847 [==============================] - 464s 597us/step - loss: 3.7645 - acc: 0.1069 - val_loss: 3.7131 - val_acc: 0.1204
Epoch 2/2
776847/776847 [==============================] - 437s 562us/step - loss: 3.6407 - acc: 0.1374 - val_loss: 3.6173 - val_acc: 0.1411


In [31]:
model.fit(x_train, y_train, validation_split=0.1, epochs=2, batch_size=128)

Train on 776847 samples, validate on 86317 samples
Epoch 1/2
776847/776847 [==============================] - 443s 570us/step - loss: 3.5505 - acc: 0.1555 - val_loss: 3.5596 - val_acc: 0.1546
Epoch 2/2
776847/776847 [==============================] - 447s 576us/step - loss: 3.4915 - acc: 0.1659 - val_loss: 3.5098 - val_acc: 0.1678


In [32]:
model.fit(x_train, y_train, validation_split=0.1, epochs=1, batch_size=128)

Train on 776847 samples, validate on 86317 samples
Epoch 1/1
776847/776847 [==============================] - 449s 578us/step - loss: 3.4495 - acc: 0.1732 - val_loss: 3.4994 - val_acc: 0.1659


In [33]:
res = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)

In [34]:
pred = model.predict(res)

In [35]:
result = np.argmax(pred, axis = 1)

In [36]:
# 写入文件
csvFile = open('C:/Users/Kai/Desktop/171840708_IntroDM_MiningChallenge/mining-challenge-for-nju-introdm-2019/Mining Challenge Dataset/1.csv','w', newline='', encoding='UTF-8') # 设置newline，否则两行之间会空一行
writer = csv.writer(csvFile)

writer.writerow(['ID', 'Expected'])
for i in range(len(result)):
    if i % 50000 == 0:
        print(i)
    writer.writerow([int(i), int(result[i])])
    
csvFile.close()

0
50000
100000
150000


In [37]:
model.save('my_model_1.h5')

In [39]:
model.fit(x_train, y_train, validation_split=0.1, epochs=1, batch_size=256)

Train on 776847 samples, validate on 86317 samples
Epoch 1/1
776847/776847 [==============================] - 232s 299us/step - loss: 3.4180 - acc: 0.1784 - val_loss: 3.4784 - val_acc: 0.1712


In [40]:
pred = model.predict(res)
result = np.argmax(pred, axis = 1)

# 写入文件
csvFile = open('C:/Users/Kai/Desktop/171840708_IntroDM_MiningChallenge/mining-challenge-for-nju-introdm-2019/Mining Challenge Dataset/2.csv','w', newline='', encoding='UTF-8') # 设置newline，否则两行之间会空一行
writer = csv.writer(csvFile)

writer.writerow(['ID', 'Expected'])
for i in range(len(result)):
    if i % 50000 == 0:
        print(i)
    writer.writerow([int(i), int(result[i])])
    
csvFile.close()

0
50000
100000
150000


In [41]:
model.save("2.h5")

In [42]:
model.fit(x_train, y_train, validation_split=0.1, epochs=1, batch_size=256)

Train on 776847 samples, validate on 86317 samples
Epoch 1/1
776847/776847 [==============================] - 250s 322us/step - loss: 3.3982 - acc: 0.1813 - val_loss: 3.4675 - val_acc: 0.1734


In [43]:
pred = model.predict(res)
result = np.argmax(pred, axis = 1)

# 写入文件
csvFile = open('C:/Users/Kai/Desktop/171840708_IntroDM_MiningChallenge/mining-challenge-for-nju-introdm-2019/Mining Challenge Dataset/3.csv','w', newline='', encoding='UTF-8') # 设置newline，否则两行之间会空一行
writer = csv.writer(csvFile)

writer.writerow(['ID', 'Expected'])
for i in range(len(result)):
    if i % 50000 == 0:
        print(i)
    writer.writerow([int(i), int(result[i])])
    
csvFile.close()

0
50000
100000
150000


In [44]:
model.fit(x_train, y_train, validation_split=0.1, epochs=1, batch_size=128)

Train on 776847 samples, validate on 86317 samples
Epoch 1/1
776847/776847 [==============================] - 505s 650us/step - loss: 3.3785 - acc: 0.1845 - val_loss: 3.4588 - val_acc: 0.1735


In [45]:
model.fit(x_train, y_train, validation_split=0.1, epochs=2, batch_size=128)

Train on 776847 samples, validate on 86317 samples
Epoch 1/2
776847/776847 [==============================] - 630s 811us/step - loss: 3.3545 - acc: 0.1882 - val_loss: 3.4532 - val_acc: 0.1756
Epoch 2/2
776847/776847 [==============================] - 606s 780us/step - loss: 3.3336 - acc: 0.1918 - val_loss: 3.4609 - val_acc: 0.1732


In [46]:
model.fit(x_train, y_train, validation_split=0.1, epochs=1, batch_size=128)

Train on 776847 samples, validate on 86317 samples
Epoch 1/1
776847/776847 [==============================] - 503s 647us/step - loss: 3.3144 - acc: 0.1953 - val_loss: 3.4599 - val_acc: 0.1736


In [47]:
model.fit(x_train, y_train, validation_split=0.1, epochs=1, batch_size=128)

Train on 776847 samples, validate on 86317 samples
Epoch 1/1
776847/776847 [==============================] - 499s 642us/step - loss: 3.2965 - acc: 0.1981 - val_loss: 3.4537 - val_acc: 0.1725


In [48]:
model.fit(x_train, y_train, validation_split=0.1, epochs=2, batch_size=128)

Train on 776847 samples, validate on 86317 samples
Epoch 1/2
776847/776847 [==============================] - 508s 654us/step - loss: 3.2792 - acc: 0.2008 - val_loss: 3.4511 - val_acc: 0.1769
Epoch 2/2
776847/776847 [==============================] - 497s 640us/step - loss: 3.2625 - acc: 0.2034 - val_loss: 3.4543 - val_acc: 0.1739


In [49]:
model.fit(x_train, y_train, validation_split=0.1, epochs=1, batch_size=512)

Train on 776847 samples, validate on 86317 samples
Epoch 1/1
776847/776847 [==============================] - 157s 202us/step - loss: 3.2409 - acc: 0.2078 - val_loss: 3.4523 - val_acc: 0.1768


In [50]:
model.fit(x_train, y_train, validation_split=0.1, epochs=1, batch_size=512)

Train on 776847 samples, validate on 86317 samples
Epoch 1/1
776847/776847 [==============================] - 179s 230us/step - loss: 3.2358 - acc: 0.2087 - val_loss: 3.4597 - val_acc: 0.1725


In [51]:
pred = model.predict(res)
result = np.argmax(pred, axis = 1)

# 写入文件
csvFile = open('C:/Users/Kai/Desktop/171840708_IntroDM_MiningChallenge/mining-challenge-for-nju-introdm-2019/Mining Challenge Dataset/3.csv','w', newline='', encoding='UTF-8') # 设置newline，否则两行之间会空一行
writer = csv.writer(csvFile)

writer.writerow(['ID', 'Expected'])
for i in range(len(result)):
    if i % 50000 == 0:
        print(i)
    writer.writerow([int(i), int(result[i])])
    
csvFile.close()

0
50000
100000
150000
